In [1]:
import mne
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd

#unfiltered_subjects = ['Aanand_19_01.csv','Abhijeet_6205876348.csv','Aanand_19_02.csv'  ,'Deepanshu_21_1.csv', 'Achal_19_01.csv', 'Adit_18_02.csv', 'Deepanshu_21_02.csv', 'Mantavya_20_01.csv', 'Mantavya_20_02.csv', 'Pritesh_22_01.csv', 'Pritesh_22_02.csv', 'Rishab_20_01.csv', 'Rishab_20_02.csv']
sfreq = 120
unfiltered_subjects = ['mohitt.csv','Aanand_19_01.csv','Abhijeet_6205876348.csv','Aanand_19_02.csv'  ,'Deepanshu_21_1.csv', 'Achal_19_01.csv', 'Adit_18_02.csv', 'Deepanshu_21_02.csv', 'Mantavya_20_01.csv', 'Mantavya_20_02.csv', 'Pritesh_22_01.csv', 'Pritesh_22_02.csv', 'Rishab_20_01.csv', 'Rishab_20_02.csv']
sfreq = 120
ch_names = ['timestamps','Fp1','Fp2','C3','C4','P7','P8','O1','O2','F7','F8','F3','F4','T7','T8','P3','P4','stim','sfreq']
#unfiltered_subjects = ['Aanand_19_02.csv']
# Create an empty DataFrame to store all data
all_data = pd.DataFrame(columns=ch_names)

for file_name in unfiltered_subjects:
    df = pd.read_csv(file_name)  # Specify index column
    df['timestamps'] = df['timestamps'] - df['timestamps'][0]
    
    lvl1_sub = df[(df['timestamps'] >= 0) & (df['timestamps'] < 150)].copy()
    lvl2_sub = df[(df['timestamps'] >= 150) & (df['timestamps'] < 280)].copy()
    lvl3_sub = df[(df['timestamps'] >= 280) & (df['timestamps'] < 400)].copy()

    lvl1_sub['label'] = 1
    lvl2_sub['label'] = 2
    lvl3_sub['label'] = 3
    
    combined_df = pd.concat([lvl1_sub, lvl2_sub, lvl3_sub])
    all_data = pd.concat([all_data, combined_df])

C:\Users\LEGION\AppData\Local\Temp\ipykernel_17940\3108172978.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data = pd.concat([all_data, combined_df])


In [3]:
drop1 = all_data.drop(['sfreq'], axis=1)
df = drop1.drop(['stim'], axis=1)


#drop_data = testing_data.drop(['sfreq'], axis=1)
# df2 = drop_data.drop(['stim'], axis=1)

In [4]:
df.shape

(681086, 18)

In [5]:
label_counts = df['label'].value_counts()
print(label_counts)

df1 = df

label
1.0    262525
2.0    227315
3.0    191246
Name: count, dtype: int64


In [6]:
df = df1.drop(['label'], axis=1)

In [7]:
eeg_data = df.iloc[:,1:18].values.T
eeg_data[:-1] = eeg_data[:-1] * (10*(-6))
eeg_data[:-1] = eeg_data[:-1] / 4

ch_names = ['Fp1',	'Fp2'	,'C3'	,'C4'	,'P7'	,'P8'	,'O1'	,'O2'	,'F7'	,'F8'	,'F3'	,'F4'	,'T7'	,'T8'	,'P3'	,'P4']
info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types = ( ["eeg" ]*16))
raw = mne.io.RawArray(eeg_data, info)
raw.set_eeg_reference()

raw.filter(l_freq = 1,h_freq = 30)

Creating RawArray with float64 data, n_channels=16, n_times=681086
    Range : 0 ... 681085 =      0.000 ...  5675.708 secs
Ready.
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 397 samples (3.308 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.4s finished


Measurement date,Unknown
Experimenter,Unknown
Participant,Unknown
Digitized points,Not available
Good channels,16 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,120.00 Hz
Highpass,1.00 Hz
Lowpass,30.00 Hz


In [8]:
duration = 0.3
epochs=mne.make_fixed_length_epochs(raw, duration = duration, overlap=0,preload=True)

Not setting metadata
18919 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 18919 events and 36 original time points ...
0 bad epochs dropped


In [9]:
epochs.drop_bad()  # This will drop bad epochs
num_epochs = len(epochs)
print("Total number of epochs created:", num_epochs)

Total number of epochs created: 18919


In [10]:
epochs.get_data()

C:\Users\LEGION\AppData\Local\Temp\ipykernel_17940\1251544707.py:1: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  epochs.get_data()


array([[[ 5.48084245e-10, -7.93058305e+05, -1.01079072e+06, ...,
         -4.48899235e+05, -4.36661523e+05, -4.23092118e+05],
        [-1.34320999e-10, -1.05370955e+06, -1.34193172e+06, ...,
         -5.89785693e+05, -5.72503401e+05, -5.56033394e+05],
        [-3.36939365e-09,  5.04360961e+06,  6.42258580e+06, ...,
          2.83214986e+06,  2.75128341e+06,  2.67096228e+06],
        ...,
        [ 1.77351467e-11, -2.25213713e+05, -2.85161659e+05, ...,
         -1.29034656e+05, -1.26277201e+05, -1.22385491e+05],
        [-1.73940862e-11,  2.84347541e+05,  3.61905087e+05, ...,
          1.60780440e+05,  1.56525691e+05,  1.51817072e+05],
        [ 5.73265879e-10, -9.24813599e+05, -1.17780133e+06, ...,
         -5.17551183e+05, -5.02400375e+05, -4.87979345e+05]],

       [[-4.10078056e+05, -3.98320784e+05, -3.85439689e+05, ...,
         -9.86561817e+04, -9.27867565e+04, -8.73434979e+04],
        [-5.39949058e+05, -5.23507607e+05, -5.07394708e+05, ...,
         -1.29568133e+05, -1.21407093e

In [11]:
import mne

eeg_data = epochs.get_data()

condition_ranges = {'lev1': (0, (num_epochs/3)),
                    'lev2': ((num_epochs/3) , 2*((num_epochs)/3) ),
                    'lev3': ( 2*((num_epochs)/3) , (num_epochs) )}

epochs_by_condition = {}
selected_epochs = []

for condition, (start_epoch, end_epoch) in condition_ranges.items():
    start_epoch = int(start_epoch)
    end_epoch = int(end_epoch)
    condition_epochs = epochs[start_epoch:end_epoch+1].copy()
    epochs_by_condition[condition] = condition_epochs
    selected_epochs.append(condition_epochs)

selected_epochs = mne.concatenate_epochs(selected_epochs)

for condition, epoch_data in epochs_by_condition.items():
    num_epochs = len(epoch_data)
    print(f"Number of epochs for {condition}: {num_epochs}")

print(f"Total number of selected epochs: {len(selected_epochs)}")


C:\Users\LEGION\AppData\Local\Temp\ipykernel_17940\3628746636.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  eeg_data = epochs.get_data()


Not setting metadata
18921 matching events found
No baseline correction applied
Number of epochs for lev1: 6307
Number of epochs for lev2: 6307
Number of epochs for lev3: 6307
Total number of selected epochs: 18921


In [12]:

condition_of_interest = 'lev1'
epoch_array_for_condition = epochs_by_condition[condition_of_interest]
print(f"Epoch array shape for condition {condition_of_interest}: {epoch_array_for_condition.get_data().shape}")


Epoch array shape for condition lev1: (6307, 16, 36)


C:\Users\LEGION\AppData\Local\Temp\ipykernel_17940\1489624503.py:3: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  print(f"Epoch array shape for condition {condition_of_interest}: {epoch_array_for_condition.get_data().shape}")


In [13]:
import pandas as pd
import numpy as np
from scipy.signal import welch



def calculate_psd_features(epoch_data, sfreq):
    psd_frequencies, psd_values = welch(epoch_data, fs=sfreq)
    psd_means = np.mean(psd_values, axis=1)  # Calculate mean PSD values for each channel
    psd_variances = np.var(psd_values, axis=1)  # Calculate variance of PSD values for each channel
    psd_std_devs = np.std(psd_values, axis=1)  # Calculate standard deviation of PSD values for each channel

    freq_min = 14
    freq_max = 30

    # Find the indices corresponding to the frequency range of interest
    idx_min = np.argmax(psd_frequencies > freq_min) - 1
    idx_max = np.argmax(psd_frequencies > freq_max)

    # Extract the PSD within the specified frequency range
    psd_values_alpha_band = psd_values[:, idx_min:idx_max]
    psd_means_alpha_band = np.mean(psd_values_alpha_band, axis=1)  # Calculate mean PSD values in alpha band for each channel
    psd_variances_alpha_band = np.var(psd_values_alpha_band, axis=1)  # Calculate variance of PSD values in alpha band for each channel
    psd_std_devs_alpha_band = np.std(psd_values_alpha_band, axis=1)  # Calculate standard deviation of PSD values in alpha band for each channel
    return psd_means_alpha_band, psd_variances_alpha_band, psd_std_devs_alpha_band

channel_names = epochs.ch_names
sfreq = epochs.info['sfreq']  # Sampling frequency

columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
columns.append('condition')
feature_df = pd.DataFrame(columns=columns)

def append_epoch_data(epoch_array, condition_label):
    global feature_df
    for epoch in epoch_array:
        psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
        epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
        epoch_features = np.append(epoch_features, condition_label)
        epoch_df = pd.DataFrame([epoch_features], columns=columns)
        feature_df = pd.concat([feature_df, epoch_df], ignore_index=True)

append_epoch_data(epochs_by_condition['lev1'], 'lev1')
append_epoch_data(epochs_by_condition['lev2'], 'lev2')
append_epoch_data(epochs_by_condition['lev3'], 'lev3')

print("Shape of feature DataFrame:", feature_df.shape)
print(feature_df.head())


c:\Users\LEGION\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 36, using nperseg = 36
  warnings.warn('nperseg = {0:d} is greater than input length '


Shape of feature DataFrame: (18921, 49)
         mean_psd_Fp1        mean_psd_Fp2         mean_psd_C3  \
0   667785.8950977427   901429.9419798586  19111588.098492227   
1  144375.61663968966  122084.48333537864  2894842.6468066033   
2   9776.730756241976  12764.887608065883   307075.7655779042   
3  19426.248287541017  5367.5272832944565   6539.258779538034   
4  3171.7542865123455   851.6716693454591   8794.820199897718   

          mean_psd_C4         mean_psd_P7         mean_psd_P8  \
0   889369.5120629362  492567.11212429666    955446.935187654   
1  114622.13557197528   67244.10099655128  125085.46874704522   
2  12907.275846631725   7222.791628839716  13727.909292572947   
3   2740.301669557562   5539.196518568037   5724.226669818963   
4  1202.3852974234937   714.8397160707312  1092.3333112609644   

          mean_psd_O1         mean_psd_O2         mean_psd_F7  \
0   899837.3992012424  1027358.6938727737  1667581.7192374172   
1  120573.13113193981   134878.5398229387  20775

In [14]:
# Rename columns using a dictionary
new_column_names = {
    'mean_psd_Fp1': 'mean_psd_Fp1',
    'mean_psd_Fp2': 'mean_psd_Fp2',
    'mean_psd_C3': 'mean_psd_C3',
    'mean_psd_C4': 'mean_psd_C4',
    'mean_psd_P7': 'mean_psd_P7',
    'mean_psd_P8': 'mean_psd_P8',
    'mean_psd_O1': 'mean_psd_O1',
    'mean_psd_O2': 'mean_psd_O2',
    'mean_psd_F7': 'mean_psd_F7',
    'mean_psd_F8': 'mean_psd_F8',
    'standarddev_psd_Fp1': 'stddev_psd_Fp1',
    'standarddev_psd_Fp2': 'stddev_psd_Fp2',
    'standarddev_psd_C3': 'stddev_psd_C3',
    'standarddev_psd_C4': 'stddev_psd_C4',
    'standarddev_psd_P7': 'stddev_psd_P7',
    'standarddev_psd_P8': 'stddev_psd_P8',
    'standarddev_psd_O1': 'stddev_psd_O1',
    'standarddev_psd_O2': 'stddev_psd_O2',
    'standarddev_psd_F7': 'stddev_psd_F7',
    'standarddev_psd_F8': 'stddev_psd_F8'
}

# Rename columns using the rename() method
feature_df.rename(columns=new_column_names, inplace=True)

# Print the DataFrame
print(feature_df.head())


         mean_psd_Fp1        mean_psd_Fp2         mean_psd_C3  \
0   667785.8950977427   901429.9419798586  19111588.098492227   
1  144375.61663968966  122084.48333537864  2894842.6468066033   
2   9776.730756241976  12764.887608065883   307075.7655779042   
3  19426.248287541017  5367.5272832944565   6539.258779538034   
4  3171.7542865123455   851.6716693454591   8794.820199897718   

          mean_psd_C4         mean_psd_P7         mean_psd_P8  \
0   889369.5120629362  492567.11212429666    955446.935187654   
1  114622.13557197528   67244.10099655128  125085.46874704522   
2  12907.275846631725   7222.791628839716  13727.909292572947   
3   2740.301669557562   5539.196518568037   5724.226669818963   
4  1202.3852974234937   714.8397160707312  1092.3333112609644   

          mean_psd_O1         mean_psd_O2         mean_psd_F7  \
0   899837.3992012424  1027358.6938727737  1667581.7192374172   
1  120573.13113193981   134878.5398229387  207755.45539593862   
2  12874.957387609154  

In [15]:
feature_df

,mean_psd_Fp1,mean_psd_Fp2,mean_psd_C3,mean_psd_C4,mean_psd_P7,mean_psd_P8,mean_psd_O1,mean_psd_O2,mean_psd_F7,mean_psd_F8,...,stddev_psd_O2,stddev_psd_F7,stddev_psd_F8,standarddev_psd_F3,standarddev_psd_F4,standarddev_psd_T7,standarddev_psd_T8,standarddev_psd_P3,standarddev_psd_P4,condition
0,667785.8950977427,901429.9419798586,19111588.098492227,889369.5120629362,492567.11212429666,955446.935187654,899837.3992012424,1027358.6938727737,1667581.7192374172,58439.25843357105,...,1328655.9152124454,2115111.924721583,40307.03483064948,69473.74849662499,48302.29935731294,592187.6816456318,78888.63289242073,53727.390721802374,908474.5131530273,lev1
1,144375.61663968966,122084.48333537864,2894842.6468066033,114622.13557197528,67244.10099655128,125085.46874704522,120573.13113193981,134878.5398229387,207755.45539593862,16114.4940826636,...,166820.2197010033,255637.5565954464,28633.728985148096,38471.472422433355,13304.80336649377,119702.14743463698,79992.75772846995,36080.91529044756,116634.54891761763,lev1
2,9776.730756241976,12764.887608065883,307075.7655779042,12907.275846631725,7222.791628839716,13727.909292572947,12874.957387609154,14487.243155863804,19372.007246364028,1233.4079943942527,...,21658.83993530823,28848.521788003916,1699.8732723636845,1802.9212146568364,1517.7451980081712,8096.705129319039,2705.930171938222,1526.0063497509934,15294.953282660968,lev1
3,19426.248287541017,5367.5272832944565,6539.258779538034,2740.301669557562,5539.196518568037,5724.226669818963,6015.273579826346,5392.039541692956,32404.695723107783,3154.3857934955613,...,5132.874125734862,27406.101302658804,2512.9002567800653,2786.8740795866415,3395.9117056975324,26056.654597340304,26313.192582999964,3002.307307031885,4984.153176421051,lev1
4,3171.7542865123455,851.6716693454591,8794.820199897718,1202.3852974234937,714.8397160707312,1092.3333112609644,997.3147734114347,642.6828355228118,4146.149667013901,347.2772149050438,...,765.4992853755875,3822.4462234972875,171.7565857031495,170.19516064344882,182.26993452334074,2039.475714622099,2358.6398635539126,358.8724907958724,890.5206398934545,lev1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18916,42146.225234099074,10568.122641439355,17108.938786540803,6712.922518560635,8968.783061059918,7582.8233763949875,9683.037778543368,12263.117815217707,4209.535990269559,1891.4125839341748,...,13539.935743287862,4647.0141674755705,1502.7892715417731,1510.086866148646,402269.2448849453,1299.1495184023734,12740.63318568474,12687.211459906906,3693.344049145851,lev3
18917,10526.451886409408,814.052424681339,521.0038685619805,1080.74227872638,412.3129195527829,647.1653630725974,1044.7419416324935,528.2947232767144,1916.6379851144575,274.8289851282165,...,528.0079634802879,1811.8294048139835,128.48990553480303,171.11700490777085,4716.360010203496,201.93142702620676,134.97094159270227,145.57889223827772,338.538325130252,lev3
18918,10086.513720312101,4893.066118942473,8634.622847319342,7395.542940630036,9376.192964286325,8415.808834957348,9361.133297394736,7828.2034244129,12572.190942797826,6367.880832686889,...,7732.4492898339695,13858.546860580762,7203.43456853213,10188.957029679692,1007305.0971915877,9726.960423855317,7779.951362068929,7844.801317431287,12669.594067333463,lev3
18919,114470.3538743852,5324.247934502671,2670.4395386049714,2945.2042492680143,2191.6537121672386,2062.3878356735727,2990.8764255675965,2308.0675221619485,12780.710010166067,3981.2886366933694,...,1828.8459093983345,23135.920966236594,6865.5916406054885,10213.332571801242,753083.265914909,11698.971744195715,5377.6692936518675,7300.348457096332,18983.000949389578,lev3


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

X = feature_df.drop(columns=['condition'])
y = feature_df['condition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8985468956406869


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

X = feature_df.drop(columns=['condition'])
y = feature_df['condition']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

gb_classifier.fit(X_train, y_train)

y_pred = gb_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
X_test_loaded = pd.read_csv('X_test.csv')
y_test_loaded = pd.read_csv('y_test.csv', header=None).values.flatten() 

In [ ]:
y_pred_inter = rf_classifier.predict(X_test_loaded)

In [ ]:
acc = accuracy_score(y_pred_inter, y_test_loaded)
print(acc)

0.9298642533936652


In [ ]:
# def append_epoch_data(epoch_array, condition_label):
#         global feature_df
#         for epoch in epoch_array:
#             psd_means, psd_variances, psd_std_devs = calculate_psd_features(epoch, sfreq)
#             epoch_features = np.concatenate([psd_means, psd_variances, psd_std_devs])
#             epoch_features = np.append(epoch_features, condition_label)
#             epoch_df = pd.DataFrame([epoch_features], columns=columns)
#             feature_df = pd.concat([feature_df, epoch_df], ignore_index=True)

# def calculate_psd_features(epoch_data, sfreq):
#         psd_frequencies, psd_values = welch(epoch_data, fs=sfreq)
#         psd_means = np.mean(psd_values, axis=1)  # Calculate mean PSD values for each channel
#         psd_variances = np.var(psd_values, axis=1)  # Calculate variance of PSD values for each channel
#         psd_std_devs = np.std(psd_values, axis=1)  # Calculate standard deviation of PSD values for each channel

#         freq_min = 14
#         freq_max = 30

#         # Find the indices corresponding to the frequency range of interest
#         idx_min = np.argmax(psd_frequencies > freq_min) - 1
#         idx_max = np.argmax(psd_frequencies > freq_max)

#         # Extract the PSD within the specified frequency range
#         psd_values_alpha_band = psd_values[:, idx_min:idx_max]
#         psd_means_alpha_band = np.mean(psd_values_alpha_band, axis=1)  # Calculate mean PSD values in alpha band for each channel
#         psd_variances_alpha_band = np.var(psd_values_alpha_band, axis=1)  # Calculate variance of PSD values in alpha band for each channel
#         psd_std_devs_alpha_band = np.std(psd_values_alpha_band, axis=1)  # Calculate standard deviation of PSD values in alpha band for each channel
#         return psd_means_alpha_band, psd_variances_alpha_band, psd_std_devs_alpha_band


In [ ]:
# import matplotlib.pyplot as plt

# # Initialize lists to store durations and accuracies
# durations = []
# accuracies = []

# # Define a range of durations to try
# duration_range = [0.1, 0.2, 0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1.0]

# for duration in duration_range:
#     epochs = mne.make_fixed_length_epochs(raw, duration=duration, overlap=0, preload=True)
    
#     num_epochs = len(epochs)

#     eeg_data = epochs.get_data()

#     # condition_ranges = {'lev1': (0, (num_epochs/3)),
#     #                     'lev2': ((num_epochs/3) , ((2*num_epochs)/3) ),
#     #                     'lev3': ((2*num_epochs)/3 , (num_epochs) )}
#     condition_ranges = {'lev1': (0, (num_epochs/6)),
#                     'lev2': (3*(num_epochs/6) , 4*((num_epochs)/6) ),
#                     'lev3': ( 5*((num_epochs)/6) , (num_epochs) )}

#     epochs_by_condition = {}
#     selected_epochs = []

#     for condition, (start_epoch, end_epoch) in condition_ranges.items():
#         start_epoch = int(start_epoch)
#         end_epoch = int(end_epoch)
#         condition_epochs = epochs[start_epoch:end_epoch+1].copy()
#         epochs_by_condition[condition] = condition_epochs
#         selected_epochs.append(condition_epochs)

#     selected_epochs = mne.concatenate_epochs(selected_epochs)
#     import pandas as pd
#     import numpy as np
#     from scipy.signal import welch



    
#     channel_names = epochs.ch_names
#     sfreq = epochs.info['sfreq']  # Sampling frequency

#     columns = [f'{feature}_{channel}' for feature in ['mean_psd', 'variance_psd', 'standarddev_psd'] for channel in channel_names]
#     columns.append('condition')
#     feature_df = pd.DataFrame(columns=columns)

    

#     append_epoch_data(epochs_by_condition['lev1'], 'lev1')
#     append_epoch_data(epochs_by_condition['lev2'], 'lev2')
#     append_epoch_data(epochs_by_condition['lev3'], 'lev3')
#     # Rename columns using a dictionary
#     new_column_names = {
#         'mean_psd_Fp1': 'mean_psd_Fp1',
#         'mean_psd_Fp2': 'mean_psd_Fp2',
#         'mean_psd_C3': 'mean_psd_C3',
#         'mean_psd_C4': 'mean_psd_C4',
#         'mean_psd_P7': 'mean_psd_P7',
#         'mean_psd_P8': 'mean_psd_P8',
#         'mean_psd_O1': 'mean_psd_O1',
#         'mean_psd_O2': 'mean_psd_O2',
#         'mean_psd_F7': 'mean_psd_F7',
#         'mean_psd_F8': 'mean_psd_F8',
#         'standarddev_psd_Fp1': 'stddev_psd_Fp1',
#         'standarddev_psd_Fp2': 'stddev_psd_Fp2',
#         'standarddev_psd_C3': 'stddev_psd_C3',
#         'standarddev_psd_C4': 'stddev_psd_C4',
#         'standarddev_psd_P7': 'stddev_psd_P7',
#         'standarddev_psd_P8': 'stddev_psd_P8',
#         'standarddev_psd_O1': 'stddev_psd_O1',
#         'standarddev_psd_O2': 'stddev_psd_O2',
#         'standarddev_psd_F7': 'stddev_psd_F7',
#         'standarddev_psd_F8': 'stddev_psd_F8'
#     }

#     # Rename columns using the rename() method
#     feature_df.rename(columns=new_column_names, inplace=True)
#     # Ensure the rest of your code is within this loop so that it executes for each duration
    
#     # Train the classifier and calculate accuracy
#     X = feature_df.drop(columns=['condition'])
#     y = feature_df['condition']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     rf_classifier.fit(X_train, y_train)
#     y_pred = rf_classifier.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
    
#     # Append duration and accuracy to lists
#     durations.append(duration)
#     accuracies.append(accuracy)

# # Plot duration vs accuracy
# plt.plot(durations, accuracies, marker='o')
# plt.xlabel('Duration')
# plt.ylabel('Accuracy')
# plt.title('Duration vs Accuracy')
# plt.grid(True)
# plt.show()
